# Stability of a Molten Shell under Applied External Pressure
Suppose we know that a molten shell is axisymmetrically indented to a depth $h$. Let's assume that the indentation profile is $z(r)$. We can construct a Monge patch over the $(r,\phi)$ plane for the surface of revolution of this profile. We want to evaluate
\begin{align}
    \mathcal{H} = \frac{\kappa}{2}\int_{\mathrm{MC}}{4\,H^2\mathrm{d}A} + \frac{\kappa}{2}\int_{\mathrm{RS}}{4\,H^2\mathrm{d}A}  + P\Delta V
\end{align}
where MC is the *Monge cap*, RS is *rest of the sphere* and
\begin{align}
    H = \frac{1}{2}\left( \kappa_1 + \kappa_2 \right)
\end{align}
and
\begin{align}
    dA = \sqrt{g}\,\mathrm{d}\theta\,\mathrm{d}r
\end{align}
Note that we want to preserve the area of the sphere before and after the indentation using a *scale transformation*. The $\Delta V$ term includes the change in volume due to the scale transformation as well.

## Bending Energy of the Spherical Part
Assuming that the Monge cap starts from polar angle, $\theta = \pi/4$, we get
\begin{align}
    \mathcal{H}_{\mathrm{RS}} &= \frac{\kappa}{2}\int_0^{2\pi}\,\int_{\pi/4}^{\pi}\,\frac{4}{R^2}\,R^2\sin\theta\,\mathrm{d}\theta\,\mathrm{d}\phi\\
    &= 4\pi\,\kappa\,\left( 1 + \frac{1}{\sqrt{2}}\right)
\end{align}

## Indentation profile
Now we will constuct a quartic polynomial indentation profile such that the profile merges smoothly with a circle in the $z-r$ plane.
Consider a quartic polynomial
\begin{align}
    z(r) &= a\,r^4 + b\,r^3 + c\,r^2 + d\,r + e \\
\end{align}
We have five boundary conditions
1. When $r=0$, $z'(0) = 0$. This gives $d = 0$.
2. When $r=0$, $z(0) = R(1 - h)$. This gives $e = R(1 - h)$ where $h$ is the ratio of indentation depth to $R$.
3. When $r=R\,\cos{\pi/4}$, $z = R\,\sin{\pi/4}$.
4. When $r=R\,\cos{\pi/4}$, $z' = -\cot{\pi/4}$.
5. When $r=R\,\cos{\pi/4}$, $z'' = -\frac{1}{R\sin^3{\pi/4}}$

We need to solve the third, fourth and fifth boundary condition equations for $a$, $b$ and $c$.

In [1]:
import sympy as sp
from IPython.display import display_latex

def disp(idx, symObj):
    eqn = '\\[' + idx + ' = ' + sp.latex(symObj) + '\\]'
    display_latex(eqn,raw=True)
    return

sp.init_printing()

a, b, c = sp.symbols('a, b, c', real=True)
h, R = sp.symbols('h, R', real=True, positive=True)
r = R*sp.S(1)/sp.sqrt(2)
eq3 = sp.Eq( a*r**4 + b*r**3 + c*r**2  + R*(1 - h) - R*sp.S(1)/sp.sqrt(2) )
eq4 = sp.Eq( 4*a*r**3 + 3*b*r**2 + 2*c*r + 1 )
eq5 = sp.Eq( (12*a*r**2 + 6*b*r + 2*c)*R*sp.S(1)/(2*sp.sqrt(2)) + 1 )
sol = sp.solve( [eq3,eq4,eq5], a, b, c)
r = sp.symbols('r', real=True, positive=True)
z = sp.collect( sol[a]*r**4 + sol[b]*r**3 + sol[c]*r**2 + R*(1 - h), r )

fz = sp.lambdify( (r,h,R), z )
fzp = sp.lambdify( (r,h,R), z.diff(r) )
fzpp = sp.lambdify( (r,h,R), z.diff(r,2) )

Thus, we have the final equation for the indentation profile

In [2]:
disp('z(r)', z)

## Area Constraint
We need to find the total surface area of the indented structure as a function of $R$. We need to find a new radius of the spherical portion $\tilde{R}$ such that the new area is same as the old area.
Area of the spherical part is obtained as follows:

In [3]:
t, p = sp.symbols('theta, phi', real=True)
As = sp.simplify( sp.integrate(R**2*sp.sin(t), (p,0,2*sp.pi), (t,sp.pi/4,sp.pi) ) ) 
disp('A_{\mathrm{RS}}', As)

Area of the Monge cap can be obtained in theory as
\begin{align}
    A_{\mathrm{MC}} = 2\pi\,\int_0^{R/\sqrt{2}}\,r\,\sqrt{1 + z'^2}\,\mathrm{d}r
\end{align}
But evaluating this expression analytically is a challenge. So we will try to numerically solve for a new radius of the sphere $\tilde{R}$ such that
\begin{align}
    (2 + \sqrt{2})\pi \tilde{R}^2 + 2\pi\,\int_0^{\tilde{R}/\sqrt{2}}\,r\,\sqrt{1 + z'^2}\,\mathrm{d}r = 4\pi R^2
\end{align}
This automatically implies that the depth of indentation $h$ will no longer be the physical depth of indentation.
It will simply become a parameter that is analogous to the indentation in the final structure.

We will do the following:
1. For a given value of $h/R$, determine the indentation profile $z(r)$.
2. Solve for $\tilde{R}$ such that the area of the indented structure is $4\pi\,R^2$.
3. Determine the new value of indentation depth $\tilde{h}$ based on $\tilde{R}$. Check if $\tilde{h} = h\,\tilde{R}/R$ preserves the shape of the profile.

In [4]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
import ipywidgets as ipw
from scipy import optimize as op
from scipy import integrate as it

def root_g(r, h, R):
    """
    Calculates square root of determinant of the metric tensor
    """
    return r*np.sqrt(1 + fzp(r, h, R)**2)

def area_diff(R1, R0, h):
    """
    Calculates the difference between areas with new radius R1 and old radius R0.
    """
    monge_area = 2*np.pi*it.quad(root_g, 0, R1*0.7071067811865475, args=(h, R1))[0]
    sphere_area = 3.414213562373095*np.pi*R1**2
    old_area = 4*np.pi*R0**2
    return monge_area + sphere_area - old_area

def rescale_radius(h, R):
    """
    Solve for new radius such that area of the indented shell is same as
    the original sphere. Then rescale the indentation as per new radius.
    """
    R1, = op.fsolve( area_diff, R, args=(R,h) )
    return R1

def deformed_half_sphere(R, h):
    theta = np.linspace(0.25*np.pi, -0.5*np.pi, 201)
    xc = R*np.cos(theta)
    yc = R*np.sin(theta)
    xd = np.linspace(0, R*0.7071067811865475, 100)
    yd = fz(xd, h, R)
    x = np.concatenate((xd, xc))
    y = np.concatenate((yd, yc))
    return x,y

Rv = 1.0
fig, ax = plt.subplots()
ax.add_artist( Circle((0,0), Rv, fill=False, ls='--') )
ax.set_aspect('equal')
ax.set_xlim(-1.5,1.5)
ax.set_ylim(-1.5,1.5)
line, = ax.plot([],[])
plt.show()

def plotSphere(h):
    R1 = rescale_radius(h, Rv)
    xr, yr = deformed_half_sphere(R1, h)
    xl = -xr[::-1]
    yl = yr[::-1]
    x = np.concatenate((xr, xl))
    y = np.concatenate((yr, yl))
    line.set_data(x,y)
    fig.canvas.draw()
    
dw = ipw.FloatSlider(value=0.5, min=0, max=1, step=0.1, description=r'$\frac{h}{R}$')
widget = ipw.interact(plotSphere, h=dw)

FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=0.5, description='$\\frac{h}{R}$', max=1.0), Output()), _dom_classes=(…

## Volume Change
We will calculate the change in volume caused by the above indentation profile. It is the volume of revolution of the following profile.

In [5]:
rn = np.linspace(0, 1/np.sqrt(2))
sphere_cap = np.sqrt(1 - rn**2)
indent_cap = fz(rn, 0.5, 1.0)
fig1, ax1 = plt.subplots()
ax1.set_aspect('equal')
ax1.set_xlim(0,1.1)
ax1.set_ylim(0,1.1)
ax1.plot( rn, sphere_cap, label=r'Sphere')
ax1.plot( rn, indent_cap, label=r'Indentation')
ax1.fill_between( rn, sphere_cap, indent_cap)
ax1.legend()
plt.show()

FigureCanvasNbAgg()

The volume of revolution of the area under indentation profile around the z-axis is

In [6]:
T = sp.symbols('Rtilde', real=True, positive=True)
zT = z.subs(R,T)
Vi = sp.simplify( 2*sp.pi*sp.integrate(r*zT, (r,0,T*1/sp.sqrt(2))) )
disp('V_i', Vi)

Volume of the spherical cap can be calculated as follows.

In [7]:
Vs = sp.simplify( 2*sp.pi*sp.integrate( r*sp.sqrt(T**2 - r**2), (r,0,T*sp.S(1)/sp.sqrt(2))) )
disp('V_s', Vs)

There is also a change in volume due to the area constraint which results in the effective shrinking of the structure. This is just the difference between volumes of spheres of radius $R$ and $\tilde{R}$.
\begin{align}
    \Delta V_{\mathrm{area\ const}} = \frac{4}{3}\pi\left( R^3 - \tilde{R}^3\right)
\end{align}

Therefore, the total change in volume is

In [8]:
dV = sp.collect( Vs - Vi + sp.Rational(4,3)*sp.pi*R**3 - sp.Rational(4,3)*sp.pi*T**3, T )
fdV = sp.lambdify( (h,T,R), dV)
disp('\Delta V', dV)

## Monge Patch Mean Curvature and Bending Energy
For a Monge patch in polar coordinates formed as a surface of revolution of a function $z(r)$, the determinant of the metric tensor is
\begin{align}
    \sqrt{g} = r\sqrt{ 1 + z'^2}
\end{align}
The principal curvatures are given by
\begin{align}
    \kappa_1 &= -\frac{z'}{r\sqrt{1 + z'^2}} = -\frac{z'}{\sqrt{g}} \\
    \kappa_2 &= -\frac{z''}{\left(1 + z'^2\right)^{3/2}} = -\frac{z''r^3}{g^{3/2}}
\end{align}
Combining the above equations we get
\begin{align}
    \mathcal{H}_{\mathrm{MC}} &= \pi\,\kappa\int_0^R{(\kappa_1 + \kappa_2)^2\sqrt{g}}\,\mathrm{d}r\\
    &= \pi\,\kappa\int_0^R{\frac{1}{r\left(1 + z'^2\right)^{5/2}}\left( rz'' + z'^3 + z'\right)^2}\,\mathrm{d}r
\end{align}
We will use numerical integration to integrate the above expression over the surface of revolution of the indentation profile.

## Numerical Evaluation of the Full Hamiltonian
Now we can calculate the full hamiltonian for a given value of $h$ and $R$.

In [9]:
from numba import jit, prange

def monge_term(r, h, R):
    """
    The integrand of Monge patch bending energy
    """
    zp = fzp(r, h, R)
    zpp = fzpp(r, h, R)
    dH = 1/(r*(1 + zp**2)**2.5)*(r*zpp + zp**3 + zp)**2
    return dH

def hamiltonian(h, R0, pressure=1.0):
    """
    H = Monge patch bending + spherical part bending energy +
                            pressure x change in volume
    R0: original radius of the sphere
    """
    R1 = rescale_radius(h, R0)
    mongeBending = np.pi*it.quad(monge_term, 0, R1*0.7071067811865475, args=(h, R1))[0]
    pdV = pressure*fdV(h, R1, R0)
    return mongeBending, pdV

Now let's plot the Hamiltonian as a function of $h$.

In [10]:
pw = ipw.FloatSlider(min=0.0, max=200.0, step=0.1, continuous_update=False)
fig2, ax2 = plt.subplots()
Hvh, = ax2.plot([],[], label=r'Total')
Mvh, = ax2.plot([],[], label=r'Indentation')
Pvh, = ax2.plot([],[], label=r'Pressure-Volume')
ax2.set_xlim(0,1.0)
ax2.set_ylim(0.0,60.0)
ax2.set_xlabel(r'h')
ax2.set_ylabel(r'$\mathcal{H}$')
ax2.legend()
plt.show()

Rv = 1.0 # The original radius of the sphere

@jit(parallel=True)
def plotH( pressure=pw ):
    N = 101
    hv = np.linspace(0.0, 1.0, N)
    Hv = np.zeros(N)
    Mv = np.zeros(N)
    Pv = np.zeros(N)
    for i in prange(N):
        Mv[i], Pv[i] = hamiltonian(hv[i], Rv, pressure)
        Hv[i] = 21.452136490675905 + Mv[i] - Pv[i]
    Hvh.set_data(hv,Hv)
    Mvh.set_data(hv,Mv)
    Pvh.set_data(hv,Pv)
    fig2.canvas.draw()

widget2 = ipw.interact(plotH)

FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='pressure', max=200.0), Outp…